# Project Advanced Lane Finding
## The goals / steps of this project are the following:


#### Camera Calibration
* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.

#### Create Binary Image for Lanes
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").

#### Detect Lane
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.

#### Output
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## Step 0. Import necessary libraries for this pipeline

In [ ]:
import numpy as np
import pickle
import cv2
import glob

from collections import deque

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import gridspec

from ipywidgets import interact, interactive, fixed
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [ ]:
# Just some help functions
def plot_comparison_images(img1,img2,title1='',title2=''):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    f.tight_layout()
    ax1.imshow(img1,cmap='gray')
    ax1.set_title(title1, fontsize=32)
    ax2.imshow(img2,cmap='gray')
    ax2.set_title(title2, fontsize=32)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
def plot_comparison_images_with_line(img1,img2,src,dst,title1='',title2=''):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    f.tight_layout()
    ax1.imshow(img1,cmap='gray')
    ax1.set_title(title1, fontsize=32)
    x1 = [src[0][0],src[1][0],src[2][0],src[3][0],src[0][0]]
    y1 = [src[0][1],src[1][1],src[2][1],src[3][1],src[0][1]]
    ax1.plot(x1, y1, 'r-',lw=4)
    
    ax2.imshow(img2,cmap='gray')
    ax2.set_title(title2, fontsize=32)
    x2 = [dst[0][0],dst[1][0],dst[2][0],dst[3][0],dst[0][0]]
    y2 = [dst[0][1],dst[1][1],dst[2][1],dst[3][1],dst[0][1]]
    ax2.plot(x2, y2, 'r-',lw=4)
#     plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Step 1. Camera Calibration & Top-Down View of Undistorted Images
### 1.1 Preparation: Find corners from chessboard images

`cv2.findChessboardCorners()` to find chessboard corners<br />

[Camera Calibration 2.4](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html)<br />
[Camera Calibration 3.0](https://docs.opencv.org/3.0-beta/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html)

---

<figure>
 <img src="materials/points mapping.jpg" width="800" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Images are distorted when taken, need to undistort them before get true features </p> 
 </figcaption>

In [ ]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
# sample chessboard images have 6x9 corners
nx = 9
ny = 6

objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
# print(objp)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')


# Loop through image and get the chessboard corners, if corners found, attach to objpoints and imgpoints
images_cannot_be_used = []
image_to_show = 12 # Sample to plot
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    origin_img = np.copy(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
    
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        corner_img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
#         write_name = 'corners_found'+str(idx)+'.jpg'
#         cv2.imwrite(write_name, img)
#         cv2.imshow('img', img)
#         cv2.waitKey(500)
    else:
        images_cannot_be_used.append(idx)
    
    # Plot a sample image
    if idx == image_to_show:
        plot_comparison_images(origin_img, corner_img, 'Original Image', 'Corners Image')

print ('Index for the calibration images that cannot be used: ',images_cannot_be_used)
# print (objp[0])
# print (imgpoints[0])

### 1.2 Get parameters for undistort imags
`cv2.calibrateCamera()` to get the matrix<br />


In [ ]:
# Try on a calibration image the function of undistortion
img = cv2.imread('./camera_cal/calibration1.jpg')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
undist = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('./result/calibration01_undist.jpg', undist)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump(dist_pickle, open( "./result/image_calibration.p", "wb" ) )
# Visualize undistortion

plot_comparison_images(img,undist,'Original Image','Undistorted Image')

print ("Ret: ", ret)
print ("Intrinsic Param: ", mtx)
print ("Extrinsic Param: ", dist)

### 1.3 Undistort a test image
[cv2.undistort()](https://docs.opencv.org/2.4/modules/imgproc/doc/geometric_transformations.html#void%20undistort(InputArray%20src,%20OutputArray%20dst,%20InputArray%20cameraMatrix,%20InputArray%20distCoeffs,%20InputArray%20newCameraMatrix)) to undistort the matrix<br />

In [ ]:
# Undistort the image, function takes RGB color image
def undistort(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)

# Load a test image
test_img = cv2.imread('./test_images/straight_lines1.jpg')
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
# plt.imshow(test_image)

# Load the parameters from the repository
# dist_param = pickle.load( open( "./result/image_calibration.p", "rb" ) )
# mtx = dist_param["mtx"]
# dist = dist_param["dist"]
# print (mtx, dist)

# Plot the comparision images
test_img_undist = undistort(test_img, mtx, dist)
plot_comparison_images(test_img, test_img_undist, 'Original Image', 'Undistorted Image')

### 1.4 Pespective transform

In [ ]:
# Define a function to perform the pespective transform
def warper(img_undist,src,dst):
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    img_size = (img_undist.shape[1], img_undist.shape[0])
    warped = cv2.warpPerspective(img_undist, M, img_size, flags=cv2.INTER_LINEAR)
    return warped, M, Minv


# Test the function on a test image
test_img = cv2.imread('./test_images/straight_lines1.jpg')
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
test_img_undist = undistort(test_img,mtx,dist)

# Should replace by video image size
test_img_size = (test_img_undist.shape[1],test_img_undist.shape[0])


src = np.float32([(580,455), (705,455), (1084,660), (235,660)])
dst = np.float32([(320,0), (test_img_size[0]-320,0), (test_img_size[0]-320,test_img_size[1]), (320,test_img_size[1])])
test_img_warp, M, Minv = warper(test_img_undist,src,dst)

dist_pickle = {}
dist_pickle["src"] = src
dist_pickle["dst"] = dst
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump(dist_pickle, open( "./result/image_calibration.p", "wb" ) )


# Plot the comparision images
plot_comparison_images_with_line(test_img_undist, test_img_warp,src,dst,'Original Image','Unwarped Image')
# plot_comparison_images(test_img_undist, test_img_unwarp,'Original Image','Unwarped Image')


In [ ]:
for test_image in glob.glob('./test_images/test*.jpg'):
    a_img = cv2.imread(test_image)
    a_img = cv2.cvtColor(a_img, cv2.COLOR_BGR2RGB)
    ud = undistort(a_img,mtx,dist)
    w, m, minv = warper(ud,src,dst)
    plot_comparison_images_with_line(ud,w,src,dst,'Original Image','Unwarped Image')

## Step 2. Binary Images for Lane Detection
### 2.1 Just visualize images at different color spaces

In [ ]:
# Just visualize images at different color spaces to get a feeling
test_img = cv2.imread('./test_images/test5.jpg')
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
test_img_undist = undistort(test_img, mtx, dist)
test_img_warp, M, Minv = warper(test_img_undist,src,dst)
plt.imshow(test_img_warp)

# RGB space
test_img_warp_R = test_img_warp[:,:,0]
test_img_warp_G = test_img_warp[:,:,1]
test_img_warp_B = test_img_warp[:,:,2]

test_img_warp_HLS = cv2.cvtColor(test_img_warp, cv2.COLOR_RGB2HLS)
test_img_warp_H = test_img_warp_HLS[:,:,0]
test_img_warp_L = test_img_warp_HLS[:,:,1]
test_img_warp_S = test_img_warp_HLS[:,:,2]

test_img_warp_LAB = cv2.cvtColor(test_img_warp, cv2.COLOR_RGB2Lab)
test_img_warp_L_2 = test_img_warp_LAB[:,:,0]
test_img_warp_A = test_img_warp_LAB[:,:,1]
test_img_warp_B_2 = test_img_warp_LAB[:,:,2]

f, ax = plt.subplots(3,3, figsize=(16, 12))
f.subplots_adjust(hspace = .2, wspace=.001)
axis = ax.ravel()
axis[0].imshow(test_img_warp_R, cmap='gray')
axis[0].set_title('RGB R-channel', fontsize=30)
axis[1].imshow(test_img_warp_G, cmap='gray')
axis[1].set_title('RGB G-Channel', fontsize=30)
axis[2].imshow(test_img_warp_B, cmap='gray')
axis[2].set_title('RGB B-channel', fontsize=30)

axis[3].imshow(test_img_warp_H, cmap='gray')
axis[3].set_title('HLS H-Channel', fontsize=30)
axis[4].imshow(test_img_warp_L, cmap='gray')
axis[4].set_title('HLS L-channel', fontsize=30)
axis[5].imshow(test_img_warp_S, cmap='gray')
axis[5].set_title('HLS S-Channel', fontsize=30)

axis[6].imshow(test_img_warp_L_2, cmap='gray')
axis[6].set_title('LAB L-channel', fontsize=30)
axis[7].imshow(test_img_warp_A, cmap='gray')
axis[7].set_title('LAB A-Channel', fontsize=30)
axis[8].imshow(test_img_warp_B_2, cmap='gray')
axis[8].set_title('LAB B-Channel', fontsize=30)


### 2.2 Applying Sobel

In [ ]:
def abs_sobel_thresh(img, orient='x', thresh_min=20, thresh_max=100):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     gray = (cv2.cvtColor(img, cv2.COLOR_RGB2Lab))[:,:,0]
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output

# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=13, mag_thresh=(20, 93)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

# Define a function to threshold an image for a given range and Sobel kernel
def dir_thresh(img, sobel_kernel=13, thresh=(0, 1.02)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output


### 2.3 Color Thresholding

In [ ]:
# Define a function that thresholds the S-channel of HLS
def hls_select(img, thresh=(160, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output

def luv_select(img, thresh=(220, 255)):
    luv = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    l_channel = luv[:,:,0]
    u_channel = luv[:,:,1]
    v_channel = luv[:,:,2]
    
    binary_output = np.zeros_like(l_channel)
    binary_output[(l_channel > thresh[0]) & (l_channel <= thresh[1])] = 1
    return binary_output

def lab_select(img, thresh=(155, 210)):
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
    l_channel = lab[:,:,0]
    a_channel = lab[:,:,1]
    b_channel = lab[:,:,2]
    
    binary_output = np.zeros_like(b_channel)
    binary_output[(b_channel > thresh[0]) & (b_channel <= thresh[1])] = 1
    return binary_output


### 2.4 Image Processing

In [ ]:
# Load the parameters from the repository
dist_param = pickle.load( open( "./result/image_calibration.p", "rb" ) )
mtx = dist_param["mtx"]
dist = dist_param["dist"]
src = dist_param["src"]
dst = dist_param["dst"]
# print (mtx, dist, dst, src)


def process_image(img):
    # Image should be in RGB format
    img_undist = cv2.undistort(img, mtx, dist, None, mtx)
    img_unwarp, img_M, img_Minv = warper(img_undist,src,dst)
    
    # Sobel Absolute
    img_sobel_abs = abs_sobel_thresh(img_unwarp)
    # Sobel Magnitude
    img_sobel_mag = mag_thresh(img_unwarp)
    # Sobel Direction
    img_sobel_dir = dir_thresh(img_unwarp)
    
    # HLS Color, using S channel
    img_hls_s = hls_select(img_unwarp)
    # LUV Color, using L channel
    img_luv_l = luv_select(img_unwarp)
    # LAB Color, using B channel
    img_lab_b = lab_select(img_unwarp)
    
    combined1 = np.zeros_like(img_sobel_abs)
    combined1[(img_sobel_mag == 1) & (img_sobel_dir == 1)] = 1
    
    combined2 = np.zeros_like(img_hls_s)
    combined2[(img_luv_l == 1) | (img_lab_b == 1)] = 1
    
    combined3 = np.zeros_like(combined1)
    combined3[(combined1 == 1) | (combined2 == 1)] = 1
    return combined2, img_Minv, img_undist

# Test all the images
images = glob.glob('./test_images/test*.jpg')

for idx, fname in enumerate(images):

    image = cv2.imread(fname)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_processed, imge_Minv, image_undist = process_image(image)
    
    plot_comparison_images(image, image_processed, fname, 'Processed')


## Step 3. Fit Lane with a Polynomial

In [ ]:
# Fit binary warped image with window sliding and polynomial
def polynomial_fit(img_warped):
    
    # Take a histogram of the bottom half of the image
    histogram = np.sum(img_warped[img_warped.shape[0]//2:,:], axis=0)

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 10
    # Set height of windows
    window_height = np.int(img_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 80
    # Set minimum number of pixels found to recenter window
    minpix = 35
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Window slider
    window_slider = []
    
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img_warped.shape[0] - (window+1)*window_height
        win_y_high = img_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        window_slider.append((win_y_low, win_y_high, win_xleft_low, win_xleft_high, win_xright_low, win_xright_high))
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    left_fit, right_fit = (None, None)
    # Fit a second order polynomial to each
    if len(leftx) != 0:
        left_fit = np.polyfit(lefty, leftx, 2)
    if len(rightx) != 0:
        right_fit = np.polyfit(righty, rightx, 2)
    
    return left_fit, right_fit, left_lane_inds, right_lane_inds, histogram, window_slider


In [ ]:
# Process Image
image_try = cv2.imread('./test_images/test4.jpg')
image_try = cv2.cvtColor(image_try, cv2.COLOR_BGR2RGB)
binary_warped, image_Minv, image_undist = process_image(image_try)
plot_comparison_images(image_undist,binary_warped,title1='Undistorted',title2='Binary')

left_fit, right_fit, left_lane_inds, right_lane_inds, histogram, window_slider = polynomial_fit(binary_warped)


In [ ]:
# Visualize data
out_img = np.uint8(np.dstack((binary_warped, binary_warped, binary_warped))*255)
for slider in window_slider:
    cv2.rectangle(out_img,(slider[2],slider[0]),(slider[3],slider[1]),(0,255,0), 2) 
    cv2.rectangle(out_img,(slider[4],slider[0]),(slider[5],slider[1]),(0,255,0), 2)

# f, (ax1,ax2) = plt.subplots(2, 1, figsize=(16, 24))
# # f.tight_layout()
# ax1.imshow(out_img)
# ax1.set_title('Unwarp Image', fontsize=24)

    
# Generate x and y values for plotting
ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

# Identify the x and y positions of all nonzero pixels in the image
nonzero = binary_warped.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

plt.figure(figsize = (9,6))
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)


In [ ]:
def polynomial_fit_with_pre_fit(img_warped, l_fit, r_fit):
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = img_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 80
    left_lane_inds = ((nonzerox > (l_fit[0]*(nonzeroy**2) + l_fit[1]*nonzeroy + l_fit[2] - margin)) & \
                      (nonzerox < (l_fit[0]*(nonzeroy**2) + l_fit[1]*nonzeroy + l_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (r_fit[0]*(nonzeroy**2) + r_fit[1]*nonzeroy + r_fit[2] - margin)) & \
                       (nonzerox < (r_fit[0]*(nonzeroy**2) + r_fit[1]*nonzeroy + r_fit[2] + margin)))  

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    left_fit, right_fit = (None, None)
    # Fit a second order polynomial to each
    if len(leftx) != 0:
        left_fit = np.polyfit(lefty, leftx, 2)
    if len(rightx) != 0:
        right_fit = np.polyfit(righty, rightx, 2)
    
    return left_fit, right_fit, left_lane_inds, right_lane_inds

In [ ]:
# Process next image
image_try_2 = cv2.imread('./test_images/test5.jpg')
image_try_2 = cv2.cvtColor(image_try_2, cv2.COLOR_BGR2RGB)
binary_warped_2, image_Minv_2, image_undist_2 = process_image(image_try_2)
plot_comparison_images(image_undist_2,binary_warped_2,title1='Undistored_2',title2='Binary_2')

left_fit2, right_fit2, left_lane_inds2, right_lane_inds2 = polynomial_fit_with_pre_fit(binary_warped_2, left_fit, right_fit )


In [ ]:
# Create an image to draw on and an image to show the selection window
out_img_2 = np.dstack((binary_warped_2, binary_warped_2, binary_warped_2))*255
window_img = np.zeros_like(out_img_2)

margin = 65

# Generate x and y values for plotting
ploty = np.linspace(0, binary_warped_2.shape[0]-1, binary_warped_2.shape[0] )
left_fitx = left_fit2[0]*ploty**2 + left_fit2[1]*ploty + left_fit2[2]
right_fitx = right_fit2[0]*ploty**2 + right_fit2[1]*ploty + right_fit2[2]

# Generate a polygon to illustrate the search window area
# And recast the x and y points into usable format for cv2.fillPoly()
left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
left_line_pts = np.hstack((left_line_window1, left_line_window2))
right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
right_line_pts = np.hstack((right_line_window1, right_line_window2))

# Draw the lane onto the warped blank image
plt.figure(figsize = (12,9))
cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
result = cv2.addWeighted(out_img_2, 1, window_img, 0.3, 0)
plt.imshow(result)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)



## Step 4. Compute the Curvature

In [ ]:
def compute_curvature(img_warped, l_fit, r_fit, l_lane_inds, r_lane_inds):
    
    # reset these values
    left_curverad, right_curverad, car_diviation = 0,0,0
    
    ploty = np.linspace(0, img_warped.shape[0]-1, img_warped.shape[0])
#     left_fitx = l_fit[0]*ploty**2 + l_fit[1]*ploty + l_fit[2]
#     right_fitx = r_fit[0]*ploty**2 + r_fit[1]*ploty + r_fit[2]
    
    nonzero = img_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Left and right lane pixel indices
    leftx = nonzerox[l_lane_inds]
    lefty = nonzeroy[l_lane_inds] 
    rightx = nonzerox[r_lane_inds]
    righty = nonzeroy[r_lane_inds]
    
    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    left_curverad = ((1 + (2*l_fit[0]*y_eval + l_fit[1])**2)**1.5) / np.absolute(2*l_fit[0])
    right_curverad = ((1 + (2*r_fit[0]*y_eval + r_fit[1])**2)**1.5) / np.absolute(2*r_fit[0])
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    
    # Car is in the center of lower y of image
    car_position = img_warped.shape[1]/2
    left_lane_in_parallel_to_car = l_fit[0]*img_warped.shape[0]**2 + l_fit[1]*img_warped.shape[0] + l_fit[2]
    right_lane_in_parallel_to_car = r_fit[0]*img_warped.shape[0]**2 + r_fit[1]*img_warped.shape[0] + r_fit[2]
    car_diviation = (car_position-(left_lane_in_parallel_to_car+right_lane_in_parallel_to_car)/2)*xm_per_pix
    
    return left_curverad, right_curverad, car_diviation
    
# a,b,c = compute_curvature(binary_warped, left_fit, right_fit, left_lane_inds, right_lane_inds)
# print(a,b,c)

## Step 6. Plotting Back onto Road

In [ ]:
def plot_on_road(img_warped, img_origin, Minv, l_fit, r_fit):

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(img_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    ploty = np.linspace(0, img_warped.shape[0]-1, img_warped.shape[0])
    left_fitx = l_fit[0]*ploty**2 + l_fit[1]*ploty + l_fit[2]
    right_fitx = r_fit[0]*ploty**2 + r_fit[1]*ploty + r_fit[2]
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img_warped.shape[1], img_warped.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(img_origin, 1, newwarp, 0.3, 0)
#     plt.imshow(result)
    return result

def plot_info(img_origin, curverad, diviation):
    img_curvature = np.copy(img_origin)
    font = cv2.FONT_HERSHEY_TRIPLEX
    text = 'Curvature: ' + '{:04.2f}'.format(curverad) + 'm'
    cv2.putText(img_curvature, text, (40,70), font, 1.5, (200,255,155), 2, cv2.LINE_AA)
    
    which_side = ''
    if diviation > 0:
        which_side = ' to the right'
    elif diviation < 0:
        which_side = 'to the left'
    abs_diviation = abs(diviation)
    text = 'Diviation: ' + '{:04.3f}'.format(abs_diviation) + 'm ' + which_side + ' of center'
    cv2.putText(img_curvature, text, (40,120), font, 1.5, (200,255,155), 2, cv2.LINE_AA)
    return img_curvature


In [ ]:
# A new image to try
image_final = cv2.imread('./test_images/test5.jpg')
image_final = cv2.cvtColor(image_final, cv2.COLOR_BGR2RGB)
binary_warped_final, image_Minv_final, image_undist_final = process_image( image_final )
left_fit_final, right_fit_final, left_lane_inds_final, right_lane_inds_final, histogram_final, window_slider_final = polynomial_fit(binary_warped_final)
left_curverad, right_curverad, car_diviation = compute_curvature( binary_warped, left_fit, right_fit, left_lane_inds, right_lane_inds )
road_plot_1 = plot_on_road( binary_warped_final, image_undist_final, image_Minv_final, left_fit_final, right_fit_final )
road_plot_2 = plot_info( road_plot_1, (left_curverad+right_curverad)/2, car_diviation )
plot_comparison_images(image_final, road_plot_2, title1='Undistorted',title2='Road')


## Step 7. Pipeline to Process Video

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self, n=5, thres=1):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        
        #average x values of the fitted line over the last n iterations
        self.bestx = None   
        
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        
        #polynomial coefficients for the most recent fit, it is a queue
        self.mem_size = n
        self.current_fit = deque(maxlen=n) #[np.array([False])] 
        
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        
        self.diffs_thres = np.array([thres/1000,thres,thres*100], dtype='float') 
    
    '''
    def eval_new_fit(self, new_fit, new_lane_inds, mem_buff=5):
        # This basical pass the new fit to output without any filtering
        self.detected = True
        self.best_fit = new_fit
        self.current_fit.append(new_fit)
    '''    
        
    def eval_new_fit(self, new_fit, new_lane_inds, mem_buff=5):
        
        # Mem_buff is the maximum number of fit we keep in the current_fit queue
        
        # If no fit lane found
        if new_fit is None:
            self.detected = False
            try:
                self.current_fit.popleft()
                if self.current_fit:
                    self.best_fit = np.average(np.asarray(self.current_fit), axis=0)
                else:
                    self.best_fit = None
            except:
                self.best_fit = None
            
            '''
            if len(self.current_fit) >= 2: # If the fit coefficient queue has more than one value
                # Update current fit queue by remove the oldest fit
                self.current_fit.popleft()
                # Best fit is the avearge over the current_fit queue
                self.best_fit = np.average(np.asarray(self.current_fit), axis=0)
            elif len(self.current_fit) == 2: # If the fit coefficient queue has only one value
                self.current_fit.popleft()
                self.best_fit = None
            else:
                self.best_fit = None
            '''
            
        else:
            # If new fit does exist
            if self.best_fit is not None:
                # Calculate change of fit coefficient in percentage, if it exceeds threshold, throw away the new fit
                self.diffs = abs(new_fit-self.best_fit) 
#                 print(new_fit)
#                 print(self.best_fit)
#                 print(self.diffs)
#                 print(self.diffs>self.diffs_thres)
            else:
                self.diffs = np.array([0,0,0], dtype='float')
            print("Difference y0,y1,y2: ", self.diffs)
            
            if self.diffs[0] > self.diffs_thres[0] or \
                self.diffs[1] > self.diffs_thres[1] or \
                self.diffs[2] > self.diffs_thres[2]:
                self.detected = False
                try:
                    self.current_fit.popleft()
                    if self.current_fit:
                        self.best_fit = np.average(np.asarray(self.current_fit), axis=0)
                    else:
                        self.best_fit = None
                except:
                    self.best_fit = None
                
            else:
                self.detected = True
                self.px_count = np.count_nonzero(new_lane_inds)
                self.current_fit.append(new_fit) # Remember current_fit is a queue with max_length = 5
                self.best_fit = np.average(np.asarray(self.current_fit), axis=0)
      
        

In [ ]:
def process_final(img):
    bin_warped, Minv, img_undist = process_image(img)
    
    # If lane is not detected in the previous frame, using polynomial_fit() to find lane, otherwise, using the previous line fit information
    if (l_lane.detected == False) or (r_lane.detected == False):
        l_fit, r_fit, l_lane_inds, r_lane_inds, _a, _b = polynomial_fit(bin_warped)
        
    else:
        l_fit, r_fit, l_lane_inds, r_lane_inds = polynomial_fit_with_pre_fit(bin_warped, l_lane.best_fit, r_lane.best_fit)
#         print ('Hist Fit', l_fit, r_fit)
    
    l_lane.eval_new_fit(l_fit, l_lane_inds)
    r_lane.eval_new_fit(r_fit, r_lane_inds)
    
    print ('Best Fit: ', l_lane.best_fit, r_lane.best_fit)
    print ('Queue Size: ', len(l_lane.current_fit), len(r_lane.current_fit) )
    
    
    
    if l_lane.best_fit is None or r_lane.best_fit is None:
        # If not fit found, display original image
        result = np.copy(img_undist)
    else:
        left_curverad, right_curverad, car_diviation = compute_curvature( bin_warped, l_lane.best_fit, r_lane.best_fit, l_lane_inds, r_lane_inds)
        plot_polyfit = plot_on_road( bin_warped, img_undist, Minv, l_lane.best_fit, r_lane.best_fit )
        result = plot_info( plot_polyfit, (left_curverad+right_curverad)/2, car_diviation )
    
    return result
    

In [ ]:
# Load some pre-computed data
dist_param = pickle.load( open( "./result/image_calibration.p", "rb" ) )
mtx = dist_param["mtx"]
dist = dist_param["dist"]
src = dist_param["src"]
dst = dist_param["dst"]

# Define new lines (lanes), memory_buff_size = 5 by default
l_lane = Line()
r_lane = Line()

video_output = 'test_videos_output/project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("test_videos/project_video.mp4")#.subclip(38,42)

# clip1 = VideoFileClip("project_video.mp4")

white_clip = clip1.fl_image(process_final)
white_clip.write_videofile(video_output, audio=False)
# %time white_clip.write_videofile(white_output, audio=False)
